In [2]:
import pandas as pd
from IPython.display import display, Markdown

display(Markdown("""
# This sheet is used to create echo-based dispensing instructions. The initial "experiment_grid" sheet in an excel book should look like this in an Excel sheet. 
# Each experiment is termed a run, but can be modified 2 blocks down if needed. PLace both this notebook and the excel file in the same directory:
"""))

# Sample experiment grid
experiment_data = {
    "Run": [1, 2, 3, 4, 5, 6],
    "Reagent 1": [150, 150, 150, 150, 150, 150],
    "Reagent 2": [0, 25, 25, 25, 25, 0],
    "Reagent 3": [100, 150, 125, 150, 25, 25],
    "Reagent 4": [150, 150, 150, 150, 150, 150],
    "Reagent 5": [450, 450, 425, 425, 450, 450],
    "Reagent 6": [150, 75, 125, 100, 200, 225]
}
experiment_df = pd.DataFrame(experiment_data)
display(experiment_df)

display(Markdown("""
# You can also add a page in your excel sheet called "reagent_concentrations" with reagent concentrations to automatically generate labels for your experiments. It should look like the one below and use exactly the labels in the experiment_grid. The page will be called reagent_concentrations:
"""))

# Sample reagent concentrations table
reagent_data = {
    'Reagent': ['Reagent 1', 'Reagent 2', 'Reagent 3', 'Reagent 4', 'Reagent 5', 'Reagent 6'],
    'Molarity(mM)': [100, 20, 100, 500, 100, 30]
}
reagent_df = pd.DataFrame(reagent_data)
display(reagent_df)


# This sheet is used to create echo-based dispensing instructions. The initial "experiment_grid" sheet in an excel book should look like this in an Excel sheet. 
# Each experiment is termed a run, but can be modified 2 blocks down if needed:


,Run,Reagent 1,Reagent 2,Reagent 3,Reagent 4,Reagent 5,Reagent 6
0,1,150,0,100,150,450,150
1,2,150,25,150,150,450,75
2,3,150,25,125,150,425,125
3,4,150,25,150,150,425,100
4,5,150,25,25,150,450,200
5,6,150,0,25,150,450,225



# You can also add a page in your excel sheet called "reagent_concentrations" with reagent concentrations to automatically generate labels for your experiments. It should look like the one below and use exactly the labels in the experiment_grid. The page will be called reagent_concentrations:


,Reagent,Molarity(mM)
0,Reagent 1,100
1,Reagent 2,20
2,Reagent 3,100
3,Reagent 4,500
4,Reagent 5,100
5,Reagent 6,30


In [ ]:
# Fill in the lines below to use this sheet to turn DOE designs or experimental designs into ECHO Sheets.
import pandas as pd
import numpy as np

plate_type_input = input("Enter the type of plate you are using (384 or 1536): ")

# Validate the input 
if plate_type_input == "384":
    plate_type = 25
elif plate_type_input == "1536":
    plate_type = 49
else:
    raise ValueError("Invalid plate type. Please enter either '384' or '1536'.")

# Add location of the excel sheet that will be used all it needs is the grid shown above:
experiment_sheet = 'name of the sheet'

# Dispensing volumes used to create the echo sheet. This will be the only part that you need to add yourself
experiment_grid = 'experiment_grid'

# Add location of the volume matrix for the plate, this is a sheet called dispensing volume, and the condition sheet for each dispensing well.
dispensing = 'dispensing_volume'
conditions = 'dispensing_conditions'

# Location of the data that will be headered and used for downstream analysis:
plate_reader_data = 'plate_reader_data'

In [ ]:
import pandas as pd

# Read the data from the Excel sheet "experiment_grid"
file_path = experiment_sheet  # Update this path to your Excel file
df = pd.read_excel(file_path, sheet_name='experiment_grid')

# Set the threshold or the amount that each well should have at max excluding dead volume (e.g., 5000)
threshold = 23000

new_df = pd.DataFrame(df['Run'])

# Function to add new columns dynamically
def add_new_column(new_df, col, suffix):
    new_col = f"{col}_{suffix}"
    new_df[new_col] = 0
    return new_col

# Split the columns and move values based on the threshold
for col in df.columns:
    if col != 'Run':
        suffix = 1
        new_col = add_new_column(new_df, col, suffix)
        total = 0
        for i, value in enumerate(df[col]):
            if pd.isna(value):  # Check for NaN (blanks) and treat them as zeros
                value = 0
            if total + value > threshold:
                suffix += 1
                new_col = add_new_column(new_df, col, suffix)
                total = 0
            new_df.at[i, new_col] = value
            total += value

with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
    new_df.to_excel(writer, sheet_name='dispensing_volume', index=False)

print("'dispensing_volume' has been added.")


'dispensing_volume' has been added.


In [ ]:
import pandas as pd

# Generate well pattern 
def generate_well_pattern(start_well, plate_type):
    rows = 'ABCDEFGHIJKLMNOP'
    cols = range(1, plate_type + 1)
    well_pattern = [f"{row}{col}" for row in rows for col in cols]
    if start_well not in well_pattern:
        raise ValueError("Invalid starting well position.")
    start_index = well_pattern.index(start_well)
    return well_pattern[start_index:] + well_pattern[:start_index]

# Get fluid types
def get_source_plate_types(sample_ids, default_type, available_types):
    plate_types = [default_type] * len(sample_ids)
    exceptions = input("Enter Sample IDs and their Source Plate Types (format: ID1:Type1, ID2:Type2,...), or press Enter to skip: ")
    if exceptions:
        for exception in exceptions.split(','):
            sample_id, plate_type = exception.split(':')
            sample_id, plate_type = sample_id.strip(), plate_type.strip()
            if sample_id in sample_ids and plate_type in available_types:
                index = sample_ids.index(sample_id)
                plate_types[index] = plate_type
            else:
                print(f"Invalid entry: {sample_id} or {plate_type} is not recognized.")
    return plate_types

# Read the data from "dispensing_volume"
file_path = experiment_sheet
df_dispensing_volume = pd.read_excel(file_path, sheet_name='dispensing_volume')
dispensing_conditions = pd.DataFrame(df_dispensing_volume.columns[1:], columns=['Sample ID'])

# Ask for the starting well
start_well = input("Enter the starting well on the Source Plate(e.g., B1): ")

# Make well pattern
well_pattern = generate_well_pattern(start_well, plate_type)
dispensing_conditions['Source Well'] = well_pattern[:len(dispensing_conditions)]

# Available fluid types
available_types = [
    '384PP_Plus_AQ', '384PP_AQ_BP', '384PP_AQ_SP', '384PP_AQ_SP_High',
    '384PP_Plus_AQ_BP', '384PP_Plus_AQ_GP', '384PP_Plus_AQ_GPSA',
    '384PP_Plus_AQ_GPSB', '384PP_Plus_AQ_SP'
]

# Get Source Plate Type
default_type = '384PP_Plus_AQ_BP'
sample_ids = dispensing_conditions['Sample ID'].tolist()
source_plate_types = get_source_plate_types(sample_ids, default_type, available_types)

# Fill 'Source Plate Type' column with the appropriate values
dispensing_conditions['Source Plate Type'] = source_plate_types
total_values = df_dispensing_volume.iloc[1:, 1:].astype(float).sum(axis=0)
total_values.name = 'Total'
dispensing_conditions['Total'] = total_values.values[:len(dispensing_conditions)]

# Print and write the resulting DataFrame for testing
print(dispensing_conditions)
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
    dispensing_conditions.to_excel(writer, sheet_name='dispensing_conditions', index=False)

print("The new sheet 'dispensing_conditions' has been added to the Excel file.")

      Sample ID Source Well Source Plate Type    Total
0   Succinate_1          A1  384PP_Plus_AQ_BP   8850.0
1         CoA_1          A2  384PP_Plus_AQ_BP    825.0
2         ATP_1          A3  384PP_Plus_AQ_BP   3275.0
3     Glycine_1          A4  384PP_Plus_AQ_BP   8850.0
4         P5P_1          A5  384PP_Plus_AQ_BP  22325.0
5         P5P_2          A6  384PP_Plus_AQ_BP   3475.0
6       Water_1          A7  384PP_Plus_AQ_BP      0.0
7     Extract_1          A8  384PP_Plus_AQ_BP  18000.0
8     Extract_2          A9  384PP_Plus_AQ_BP  21000.0
9     Extract_3         A10  384PP_Plus_AQ_BP  21000.0
10    Extract_4         A11  384PP_Plus_AQ_BP  21000.0
11    Extract_5         A12  384PP_Plus_AQ_BP  21000.0
12    Extract_6         A13  384PP_Plus_AQ_BP  21000.0
13    Extract_7         A14  384PP_Plus_AQ_BP  21000.0
14    Extract_8         A15  384PP_Plus_AQ_BP  21000.0
15    Extract_9         A16  384PP_Plus_AQ_BP  12000.0
The new sheet 'dispensing_conditions' has been added to the Excel

In [ ]:
import pandas as pd

def well_to_index(start_well, plate_type):
    # Split the input into row and column
    row = start_well[:-2].upper() if len(start_well) > 2 else start_well[:-1].upper()
    col = int(start_well[-2:]) if len(start_well) > 2 else int(start_well[-1:])
    
    # List of rows from A to Z and AA to ZZ
    rows = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + [chr(i) + chr(j) for i in range(65, 91) for j in range(65, 91)]
    
    row_index = rows.index(row)
    index = row_index * plate_type + (col - 1)  # Adjust to 0-based index
    return index

def run_to_well(run, start_well='A1', plate_type=48):
    rows = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + [chr(i) + chr(j) for i in range(65, 91) for j in range(65, 91)]
    cols = range(1, plate_type + 1)
    wells = [f'{r}{c}' for r in rows for c in cols]
    
    start_index = well_to_index(start_well, plate_type)
    return wells[run + start_index - 1]  # Adjusted for 0-based indexing

start_well = input("Please enter the starting well on your source plate (e.g., A1): ")
plate_type = 48  # Assuming the plate type is 48

df = pd.read_excel(experiment_sheet, sheet_name=dispensing).transpose().reset_index()
df.columns = df.iloc[0]
df = df[1:]
df.rename(columns={'Run': 'Sample ID'}, inplace=True)
df_source_data = pd.read_excel(experiment_sheet, sheet_name=conditions)
df_melted = df.melt(id_vars='Sample ID', var_name='Run', value_name='Transfer Volume')
df_melted['Run'] = pd.to_numeric(df_melted['Run'])
df_melted['Destination Well'] = df_melted['Run'].apply(lambda x: run_to_well(x, start_well=start_well, plate_type=plate_type))
df_melted = pd.merge(df_melted, df_source_data, on='Sample ID', how='left')

# Remove rows where Transfer Volume is 0
df_melted = df_melted[df_melted['Transfer Volume'] != 0]

df_melted['Source Plate'] = 'Source[1]'
df_melted['Destination Plate Name'] = '384PP_Dest'
df_melted['Destination Well X Offset'] = df_melted.get('Destination Well X Offset', 0)
df_melted['Destination Well Y Offset'] = df_melted.get('Destination Well Y Offset', 0)

df_melted = df_melted[['Source Plate', 'Source Plate Type', 'Source Well', 'Sample ID', 
                       'Destination Plate Name', 'Destination Well', 'Transfer Volume',
                       'Destination Well X Offset', 'Destination Well Y Offset']]

output_file_path = experiment_sheet
new_sheet_name = 'ECHO_SHEET'

with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_melted.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(df_melted)

    Source Plate Source Plate Type Source Well    Sample ID  \
0      Source[1]  384PP_Plus_AQ_BP          A1  Succinate_1   
2      Source[1]  384PP_Plus_AQ_BP          A3        ATP_1   
3      Source[1]  384PP_Plus_AQ_BP          A4    Glycine_1   
4      Source[1]  384PP_Plus_AQ_BP          A5        P5P_1   
7      Source[1]  384PP_Plus_AQ_BP          A8    Extract_1   
..           ...               ...         ...          ...   
944    Source[1]  384PP_Plus_AQ_BP          A1  Succinate_1   
946    Source[1]  384PP_Plus_AQ_BP          A3        ATP_1   
947    Source[1]  384PP_Plus_AQ_BP          A4    Glycine_1   
949    Source[1]  384PP_Plus_AQ_BP          A6        P5P_2   
959    Source[1]  384PP_Plus_AQ_BP         A16    Extract_9   

    Destination Plate Name Destination Well  Transfer Volume  \
0               384PP_Dest               A1              150   
2               384PP_Dest               A1              100   
3               384PP_Dest               A1        

In [7]:
import pandas as pd

# Load the data making sure there is a "reagent_concentrations" sheet
def load_data(file_path):
    experiment_grid = pd.read_excel(file_path, sheet_name='experiment_grid')
    reagent_concentrations = pd.read_excel(file_path, sheet_name='reagent_concentrations')
    return experiment_grid, reagent_concentrations

# Convert volumes to molarities
def convert_to_molarities(experiment_grid, reagent_concentrations, reaction_volume_ul):
    molarities = experiment_grid.copy()
    
    # Iterate over the columns to convert volumes to molarities
    for reagent in reagent_concentrations['Reagent']:
        if reagent in molarities.columns:
            concentration = reagent_concentrations.loc[reagent_concentrations['Reagent'] == reagent, 'Molarity(mM)'].values[0]
            # Convert volume from nL to µL by dividing by 1000, then calculate molarity
            molarities[reagent] = (molarities[reagent] / (reaction_volume_ul * 1000)) * concentration
    
    return molarities

# Main function
def main():
    # Ask the user for the volume of the reaction
    reaction_volume_ul = float(input("Enter the volume of the reaction (in µL): "))
    
    # Load the data
    file_path = experiment_sheet
    experiment_grid, reagent_concentrations = load_data(file_path)
    
    # Convert volumes to molarities
    molarities = convert_to_molarities(experiment_grid, reagent_concentrations, reaction_volume_ul)
    
    # Load the existing Excel file
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        # Save the molarities to a new sheet called 'dispensed_concentrations'
        molarities.to_excel(writer, sheet_name='dispensed_concentrations', index=False)
    
    print(f"Molarities have been saved to the 'dispensed_concentrations' sheet in {file_path}")

if __name__ == "__main__":
    main()

Molarities have been saved to the 'dispensed_concentrations' sheet in 12_22_24_PPIX_RF_opts_2.xlsx


In [ ]:
import openpyxl


file_path = experiment_sheet  
wb = openpyxl.load_workbook(file_path)

# Read reagents from the first row
sheet = wb[experiment_grid]
reagents = [cell.value for cell in sheet[1]][1:]

# Read data from the sheet
data = {}
for row in sheet.iter_rows(min_row=2, values_only=True):  # Skip the first row (headers)
    test = row[0]
    values = list(row[1:])  # Skip the first column (test names)
    data[test] = values

# Create headers
headers = {}
for test, values in data.items():
    headers[test] = []
    for i, value in enumerate(values):
        if value != 0 and value is not None:  # Check for non-zero and non-None values
            headers[test].append(f"{reagents[i]}:{value}")

# Convert list of headers
output = []
for test, values in headers.items():
    if values:  # Only add to output if there are headers
        headers_string = ', '.join(values)
    else:
        headers_string = 'None'  # Add 'None' if there are no headers
    output.append(headers_string)  # Removed test name from output

if plate_reader_data in wb.sheetnames:
    sheet = wb[plate_reader_data]
else:
    sheet = wb.create_sheet(title=plate_reader_data)

for row in sheet.iter_rows():
    for cell in row:
        cell.value = None

for i, header in enumerate(output, start=2):  
    sheet.cell(row=1, column=i, value=header)

# Save the workbook
wb.save(file_path)

print("The new sheet 'plate_reader_data' has been added to the Excel file with headers.")


The new sheet 'plate_reader_data' has been added to the Excel file.


In [17]:
# This script creates a visual representation of a plate map with sample color codes and overlays pie charts on the correct map.

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from collections import defaultdict

# Load the Excel file
file_path = '12_22_24_PPIX_RF_opts_2.xlsx'
sheet_name = 'ECHO_SHEET'

# Read the data from the specified sheet
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Print the data to understand its structure
print(df.head())

# Get unique sample IDs and assign colors dynamically
unique_sample_ids = df['Sample ID'].dropna().unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_sample_ids)))
color_code_mapping = {sample_id: color for sample_id, color in zip(unique_sample_ids, colors)}

# Map the color codes to the DataFrame
df['Color Code'] = df['Sample ID'].map(color_code_mapping)

filtered_df = df[df['Color Code'].notna()]

# Determine the plate type 
max_row = filtered_df['Destination Well'].str[0].map(ord).max() - 64 # Convert letter to number (A=1, B=2, etc.)
max_col = filtered_df['Destination Well'].str[1:].astype(int).max()

if max_row <= 16 and max_col <= 24:
    plate_type = '384 Well Plate'
    rows, cols = 16, 24
elif max_row <= 32 and max_col <= 48:
    plate_type = '1536 Well Plate'
    rows, cols = 32, 48
else:
    raise ValueError('The data does not match the dimensions of a 384 or 1536 well plate.')

print(f"Detected plate type: {plate_type}")

# Group data by destination well
well_data = defaultdict(list)
for _, row in filtered_df.iterrows():
    well_data[row['Destination Well']].append(row['Color Code'])

# Create a visual representation of the plate map
fig, ax = plt.subplots(figsize=(cols / 2, rows / 2))

# Set up the grid
ax.set_xticks(np.arange(1, cols + 1, 1))
ax.set_yticks(np.arange(1, rows + 1, 1))
ax.set_xticklabels(range(1, cols + 1))
ax.set_yticklabels([chr(i) for i in range(65, 65 + rows)])
ax.set_xlim(0.5, cols + 0.5)
ax.set_ylim(0.5, rows + 0.5)
ax.grid(True, which='both', linestyle='-', color='gray', linewidth=0.5)

for row in range(1, rows + 1):
    for col in range(1, cols + 1):
        well = f"{chr(64 + row)}{col}"
        if well not in well_data:
            well_data[well] = []

# Plot each well with its corresponding color slices
for well, colors in well_data.items():
    dest_row = ord(well[0]) - 65 + 1  # Convert letter to number (A=1, B=2, etc.)
    dest_col = int(well[1:])
    if dest_row <= rows and dest_col <= cols:
        sizes = [1] * len(colors) if colors else [1]  # Equal size for each slice, or a blank slice for empty wells
        wedges, _ = ax.pie(sizes, colors=colors if colors else ['white'], radius=0.4, center=(dest_col, dest_row), wedgeprops=dict(width=0.4))

handles = [mpatches.Patch(color=color, label=sample_id) for sample_id, color in color_code_mapping.items()]
ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xlabel('Columns')
ax.set_ylabel('Rows')
ax.set_title(f'{plate_type} Map with Sample Color Codes')

ax.invert_yaxis()

plt.subplots_adjust(left=0.1, right=0.85, top=0.95, bottom=0.1)

ax.tick_params(axis='both', which='both', length=6, width=2, colors='black', grid_color='gray', grid_alpha=0.5)

plt.show()

  Source Plate Source Plate Type Source Well    Sample ID  \
0    Source[1]  384PP_Plus_AQ_BP          A1  Succinate_1   
1    Source[1]  384PP_Plus_AQ_BP          A3        ATP_1   
2    Source[1]  384PP_Plus_AQ_BP          A4    Glycine_1   
3    Source[1]  384PP_Plus_AQ_BP          A5        P5P_1   
4    Source[1]  384PP_Plus_AQ_BP          A8    Extract_1   

  Destination Plate Name Destination Well  Transfer Volume  \
0             384PP_Dest               A1              150   
1             384PP_Dest               A1              100   
2             384PP_Dest               A1              150   
3             384PP_Dest               A1              450   
4             384PP_Dest               A1             3000   

   Destination Well X Offset  Destination Well Y Offset  
0                          0                          0  
1                          0                          0  
2                          0                          0  
3                          0